In [1]:
# from selenium import webdriver
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.common import action_chains, keys
# from selenium import webdriver
# from selenium.webdriver.common.action_chains import ActionChains
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

# from pyvirtualdisplay import Display
# import gnp
# import codecs
# import json
# from boto.s3.connection import S3Connection
# from boto.s3.key import Key
# import boto
import sys, os
import pandas as pd
# from TwitterSearch import *
# from sunlight import congress
# from bs4 import BeautifulSoup
# import urllib2
# import itertools
# import datetime
# import configparser
# import time
from pattern.en import parsetree
from pattern.en import mood
import dateutil.parser as dparser
import sunlight
from sunlight import congress
import re
from postal.parser import parse_address
import csv
import selenium
from bs4 import BeautifulSoup as soupy
import tweepy
import codecs
import io
import phonenumbers
import re
import editdistance
import nltk
from geopy.geocoders import Nominatim
import address
from address import AddressParser, Address
# import geotext
# from geotext import GeoText
# import geograpy
# import pprint
# import ner
# from nltk.tag import StanfordNERTagger
# from nltk.tokenize import word_tokenize
# import editdistance
import calendar
# from itertools import islice
# from nltk.chunk import conlltags2tree, tree2conlltags, ne_chunk
# import nltk


# def take(n, iterable):
#     "Return first n items of the iterable as a list"
#     return list(islice(iterable, n))

import spacy

In [25]:
actionability_train = df = pd.read_csv('~/w210_imwithdata/data/static_data/full_rzst_test_dataset.csv')

In [3]:
# import sys  

# reload(sys)  
# sys.setdefaultencoding('utf8')

In [26]:
actionability_train[:10]

,issue,action,id,es_score,tweet_timestamp,query_timestamp,tweet_user,tweet,actionable
0,healthcare,charity,AVrp-o7QMNAj8foZ-YaR,8.655990,2017-03-20T04:24:54.000Z,2017-03-19T23:37:51.916551,AnwaraliAnwaar,https://t.co/9vAellIc6i - RSS Channel: Colin K...,0
1,healthcare,charity,AVrpI7X7MNAj8foZ-Sjm,7.009409,2017-03-20T00:30:14.000Z,2017-03-19T23:37:51.916551,CrowdExposure,Please Support @kevinsanders23 Baby Bombers an...,1
2,healthcare,charity,AVrpM21hMNAj8foZ-TFR,7.009409,2017-03-20T00:47:24.000Z,2017-03-19T23:37:51.916551,KimothyJoy,Reminder that the GOP alternative to the Affor...,1
3,healthcare,charity,AVrpDUi3MNAj8foZ-Rx5,6.756997,2017-03-20T00:05:45.000Z,2017-03-19T23:37:51.916551,RussWhitworth,Don't forget to donate accordingly to https://...,1
4,healthcare,charity,AVrpsYiZMNAj8foZ-Ws5,6.756997,2017-03-20T03:05:08.000Z,2017-03-19T23:37:51.916551,TheRealLilXza,I'm raising money for ACA- Affordable Care Act...,1
5,healthcare,charity,AVrqSJjVMNAj8foZ-ZrD,6.756997,2017-03-20T05:50:08.000Z,2017-03-19T23:37:51.916551,Clearlease,Please Support @kevinsanders23 Baby Bombers an...,1
6,healthcare,charity,AVrpRRd9MNAj8foZ-Tkx,6.750567,2017-03-20T01:06:41.000Z,2017-03-19T23:37:51.916551,PawWorks,ALERT: Violet needs help!\r\rShe is at VCA Wes...,1
7,healthcare,charity,AVrpSkdTMNAj8foZ-Ttg,6.750567,2017-03-20T01:12:22.000Z,2017-03-19T23:37:51.916551,WineGumLady,@ConleyU @StephenKing we can't control who th...,0
8,healthcare,charity,AVrpVvedMNAj8foZ-UFu,6.750567,2017-03-20T01:26:13.000Z,2017-03-19T23:37:51.916551,JelliDonut,I will donate to your opposition. https://t.co...,0
9,healthcare,charity,AVrqBKmcMNAj8foZ-YmU,6.750567,2017-03-20T04:35:56.000Z,2017-03-19T23:37:51.916551,griffin_ghia,Say NO to Trumpcare and YES to the Affordable ...,1


In [27]:
tweet_text_list = actionability_train['tweet'].tolist()
tweet_text_list[:10]


['https://t.co/9vAellIc6i - RSS Channel: Colin Kaepernick pledges to donate money fromjersey sales https://t.co/mFnw8q7tyl',
 'Please Support @kevinsanders23 Baby Bombers and Medicare Donate Here - https://t.co/HLliHGEpNF ID=4',
 'Reminder that the GOP alternative to the Affordable Care Act defunds @ppfa _\xd9\xf7_Donate now and show\x89\xdb_ https://t.co/3aWSX3tXBb',
 "Don't forget to donate accordingly to https://t.co/pSpQmWIGV1. The AARP spends more than $7 million/yr lobbying fed gvt. to protect Medicare",
 "I'm raising money for ACA- Affordable Care Act Fund. Click to Donate:  https://t.co/9ymuQZLZbc via @gofundme",
 'Please Support @kevinsanders23 Baby Bombers and Medicare Donate Here - https://t.co/szNPbaCXIT ID=5',
 'ALERT: Violet needs help!\r\rShe is at VCA Westlake now with a prolapsed rectum for the third time\r\rDonate at\x89\xdb_ https://t.co/0BYOk23LkA',
 "@ConleyU  @StephenKing we can't control who they choose to donate to. should people/kids die/go hungry if they didn'

In [28]:
all_tweet_phones = []

for tweet in tweet_text_list:
    matches = []
    if phonenumbers.PhoneNumberMatcher(tweet.decode('latin-1'), "US"):
        for match in phonenumbers.PhoneNumberMatcher(tweet.decode('latin-1'), "US"):
            matches.append(phonenumbers.format_number(match.number, phonenumbers.PhoneNumberFormat.NATIONAL))
        all_tweet_phones.append(matches)
    else:
        all_tweet_phones.append([])


print all_tweet_phones[:100]
        

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [29]:
all_tweet_urls = []

# ANY_URL_REGEX = r"""(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))"""
WEB_URL_REGEX = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""

for tweet in tweet_text_list:
    matches = []
    if re.findall(WEB_URL_REGEX,tweet):
        all_tweet_urls.append(re.findall(WEB_URL_REGEX,tweet))
    else:
        all_tweet_urls.append([])
        



In [30]:
all_tweet_urls[:10]

[['https://t.co/9vAellIc6i', 'https://t.co/mFnw8q7tyl'],
 ['https://t.co/HLliHGEpNF'],
 ['https://t.co/3aWSX3tXBb'],
 ['https://t.co/pSpQmWIGV1'],
 ['https://t.co/9ymuQZLZbc'],
 ['https://t.co/szNPbaCXIT'],
 ['https://t.co/0BYOk23LkA'],
 [],
 ['https://t.co/01w9JjVg23'],
 ['https://t.co/rJraok6mOf']]

In [ ]:
# all_tweet_city_state = []

# for tweet in tweet_text_list:
#     try:
#         places = geograpy.get_place_context(text = tweet)
#         if places.city_mentions:
#             print places.city_mentions
#         if places.region_mentions:
#             print places.region_mentions
#     except:
#         pass
#     else:
#         all_tweet_urls.append([])
        

In [ ]:
# geolocator = Nominatim()


In [20]:
# for urls in all_tweet_urls:
#     if urls:
#         try:
#             places = geograpy.get_place_context(url = urls[0])
#             if places.city_mentions:
#                 print places.city_mentions
#             if places.region_mentions:
#                 print places.region_mentions
#         except:
#             pass

In [5]:
# print os.environ.get('STANFORD_MODELS','/etc/environment')

# # Output:
# #/home/jackspicer/StanfordParser/stanford-ner-2015-12-09/classifiers/english.all.3class.distsim.crf.ser.gz:/home/jackspicer/stanford-postagger-2015-12-09/models/english.conll.4class.distsim.crf.ser.gz:/home/jackspicer/StanfordParser/stanford-ner-2015-12-09/english.muc.7class.distsim.crf.ser.gz

# # I want to select the first classifier model

# stanford_classifier = os.environ.get('STANFORD_MODELS','/etc/environment').split(':')[0]

# # For getting the path for StanfordNERTagger 
# stanford_ner_path = os.environ.get('CLASSPATH','/etc/environment').split(':')[0]

# st = StanfordNERTagger(stanford_classifier, stanford_ner_path, encoding='utf-8')

# print st.tag("Manash is awesome!".split())

In [10]:
tweet_text_list[0]

'https://t.co/9vAellIc6i - RSS Channel: Colin Kaepernick pledges to donate money fromjersey sales https://t.co/mFnw8q7tyl'

In [11]:
tweet_text_list[1]

'Please Support @kevinsanders23 Baby Bombers and Medicare Donate Here - https://t.co/HLliHGEpNF ID=4'

In [12]:
int(editdistance.eval(tweet_text_list[0],tweet_text_list[1]))

90

In [39]:
sorted([3,1,2])

[1, 2, 3]

In [46]:
distance_dict = {}

for i, tweet in enumerate(tweet_text_list[:1000]):
    for j, tweet2 in enumerate(tweet_text_list[:1000]):
        tweet_ids = tuple(sorted([i,j]))
        distance_dict[tweet_ids] = int(editdistance.eval(tweet,tweet2))

In [52]:
print take(100, distance_dict.iteritems())

[((668, 732), 109), ((381, 392), 88), ((703, 837), 102), ((259, 938), 119), ((432, 939), 107), ((150, 805), 117), ((337, 866), 107), ((247, 483), 120), ((284, 367), 113), ((747, 795), 119), ((648, 678), 106), ((0, 862), 117), ((203, 889), 119), ((613, 958), 113), ((25, 701), 99), ((544, 667), 122), ((228, 412), 120), ((122, 635), 116), ((69, 912), 112), ((703, 896), 109), ((290, 555), 113), ((94, 509), 104), ((424, 521), 101), ((209, 569), 118), ((262, 589), 109), ((156, 192), 95), ((359, 971), 100), ((234, 924), 115), ((181, 691), 114), ((75, 400), 120), ((22, 761), 49), ((367, 853), 116), ((522, 721), 119), ((100, 805), 107), ((619, 735), 117), ((47, 798), 116), ((453, 982), 104), ((56, 323), 108), ((251, 961), 115), ((296, 338), 84), ((499, 877), 116), ((812, 906), 121), ((650, 869), 87), ((134, 666), 96), ((28, 185), 93), ((321, 929), 101), ((493, 959), 120), ((268, 744), 89), ((53, 538), 101), ((346, 532), 120), ((581, 883), 110), ((187, 536), 115), ((553, 879), 100), ((78, 967), 

In [60]:
all_tweet_dates = []

for tweet in tweet_text_list:
    matches = []
    try:
        if dparser.parse(tweet,fuzzy=False):
            all_tweet_dates.append(dparser.parse(tweet,fuzzy=False))
        else:
            all_tweet_dates.append([])
    except:
        all_tweet_dates.append([])
        

In [70]:
# print all_tweet_dates[2000:5000]
# print tweet_text_list[:100]
print ('|').join(calendar.month_name[1:13])
print ('|').join(calendar.month_abbr[1:13])
calendar.month_abbr[1:13]


January|February|March|April|May|June|July|August|September|October|November|December
Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec


['Jan',
 'Feb',
 'Mar',
 'Apr',
 'May',
 'Jun',
 'Jul',
 'Aug',
 'Sep',
 'Oct',
 'Nov',
 'Dec']

In [107]:
all_tweet_dates = []

for tweet in tweet_text_list:
    matches = []
    if re.findall(r'(\d{2})[/.-](\d{2})[/.-](\d{4})$', tweet):
        matches.append(re.findall(r'(\d{2})[/.-](\d{2})[/.-](\d{4})$', tweet))
    if re.findall(r'(\d{2})[/.-](\d{2})$', tweet):
        matches.append(re.findall(r'(\d{2})[/.-](\d{2})$', tweet))
    if re.findall(r'\d\d\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\d{2}\d{4}', tweet):
        matches.append(re.findall(r'\d\d\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\d{2}\d{4}', tweet))
    if re.findall(r'(?:(September|April|June|November|Sep|Apr|Jun|Nov) +(0?[1-9]|[12]\d|30), *((?:19|20)\d\d))|(?:(January|March|May|July|August|October|December|Jan|Mar|May|Jul|Aug|Sep|Oct|Nov|Dec) +(0?[1-9]|[12]\d|3[01]), *((?:19|20)\d\d))|(?:(February|Feb) +(?:(?:(0?[1-9]|1\d|2[0-8]), *((?:19|20)\d\d))|(?:(29), *((?:(?:19|20)(?:04|08|12|16|20|24|28|32|36|40|44|48|52|56|60|64|68|72|76|80|84|88|92|96))|2000))))',tweet):
        matches.append(re.findall(r'(?:(September|April|June|November|Sep|Apr|Jun|Nov) +(0?[1-9]|[12]\d|30), *((?:19|20)\d\d))|(?:(January|March|May|July|August|October|December|Jan|Mar|May|Jul|Aug|Sep|Oct|Nov|Dec) +(0?[1-9]|[12]\d|3[01]), *((?:19|20)\d\d))|(?:(February|Feb) +(?:(?:(0?[1-9]|1\d|2[0-8]), *((?:19|20)\d\d))|(?:(29), *((?:(?:19|20)(?:04|08|12|16|20|24|28|32|36|40|44|48|52|56|60|64|68|72|76|80|84|88|92|96))|2000))))',tweet))
    elif re.findall(r'(?:(September|April|June|November|Sep|Apr|Jun|Nov) +(0?[1-9]|[12]\d|30))|(?:(January|March|May|July|August|October|December|Jan|Mar|May|Jul|Aug|Sep|Oct|Nov|Dec) +(0?[1-9]|[12]\d|3[01]))|(?:(February|Feb) +(?:(?:(0?[1-9]|1\d|2[0-8]))|(?:(29), *((?:(?:19|20)(?:04|08|12|16|20|24|28|32|36|40|44|48|52|56|60|64|68|72|76|80|84|88|92|96))|2000))))',tweet):
        matches.append(re.findall(r'(?:(September|April|June|November|Sep|Apr|Jun|Nov) +(0?[1-9]|[12]\d|30))|(?:(January|March|May|July|August|October|December|Jan|Mar|May|Jul|Aug|Sep|Oct|Nov|Dec) +(0?[1-9]|[12]\d|3[01]))|(?:(February|Feb) +(?:(?:(0?[1-9]|1\d|2[0-8]))|(?:(29), *((?:(?:19|20)(?:04|08|12|16|20|24|28|32|36|40|44|48|52|56|60|64|68|72|76|80|84|88|92|96))|2000))))',tweet))
    if re.findall(r'(Jan(uary)?|Feb(ruary)?|Mar(ch)?|Apr(il)?|May|Jun(e)?|Jul(y)?|Aug(ust)?|Sep(tember)?|Oct(ober)?|Nov(ember)?|Dec(ember)?)\s+\d{1,2},\s+\d{4}',tweet):
        matches.append(re.findall(r'(Jan(uary)?|Feb(ruary)?|Mar(ch)?|Apr(il)?|May|Jun(e)?|Jul(y)?|Aug(ust)?|Sep(tember)?|Oct(ober)?|Nov(ember)?|Dec(ember)?)\s+\d{1,2},\s+\d{4}',tweet))
    if re.findall(r'[0-9]{2}/[0-9]{2}/[0-9]{4}',tweet):
        matches.append(re.findall(r'[0-9]{2}/[0-9]{2}/[0-9]{4}',tweet))
    if re.findall(r'[0-9]{2}/[0-9]{2}',tweet):
        matches.append(re.findall(r'[0-9]{2}/[0-9]{2}',tweet))
    if re.findall(r'[0-9]{2}/[0-9]{1}',tweet):
        matches.append(re.findall(r'[0-9]{2}/[0-9]{1}',tweet))
#     if re.findall(r'\w{3,9}?\s\d{1,2}?\s,\s\d{4}?',tweet):
#         matches.append(re.findall(r'\w{3,9}?\s\d{1,2}?\s,\s\d{4}?',tweet))
    all_tweet_dates.append(matches)
        

# re.match(r'(\d{2})[/.-](\d{2})[/.-](\d{4})$', tweet)
# re.match(r'(\d{2})[/.-](\d{2})$', tweet)
# re.match(r'\d\d\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{4}', text)
# re.match(r'(?:(September|April|June|November|Sep|Apr|Jun|Nov) +(0?[1-9]|[12]\d|30), *((?:19|20)\d\d))|(?:(January|March|May|July|August|October|December|Jan|Mar|May|Jul|Aug|Sep|Oct|Nov|Dec) +(0?[1-9]|[12]\d|3[01]), *((?:19|20)\d\d))|(?:(February|Feb) +(?:(?:(0?[1-9]|1\d|2[0-8]), *((?:19|20)\d\d))|(?:(29), *((?:(?:19|20)(?:04|08|12|16|20|24|28|32|36|40|44|48|52|56|60|64|68|72|76|80|84|88|92|96))|2000))))',tweet)
# re.match(r'(Jan(uary)?|Feb(ruary)?|Mar(ch)?|Apr(il)?|May|Jun(e)?|Jul(y)?|Aug(ust)?|Sep(tember)?|Oct(ober)?|Nov(ember)?|Dec(ember)?)\s+\d{1,2},\s+\d{4}',tweet)
# re.match(r'^[a-zA-Z]{3}\s+\d{1,2}\s',tweet)
# re.match(r'\w{3,9}?\s\d{1,2}?\s,\s\d{4}?',tweet)


In [2]:
# for i,dated in enumerate(all_tweet_dates):
#     if len(dated) > 0:
#         print i, ' ', dated, tweet_text_list[i]

In [25]:
sentence = "I am going to Biloxi, Mississippi."
ne_tree = ne_chunk(nltk.pos_tag(word_tokenize(sentence)))
 
iob_tagged = tree2conlltags(ne_tree)
print iob_tagged

 
ne_tree = conlltags2tree(iob_tagged)
print ne_tree

[('I', 'PRP', u'O'), ('am', 'VBP', u'O'), ('going', 'VBG', u'O'), ('to', 'TO', u'O'), ('Biloxi', 'NNP', u'B-GPE'), (',', ',', u'O'), ('Mississippi', 'NNP', u'B-GPE'), ('.', '.', u'O')]
(S
  I/PRP
  am/VBP
  going/VBG
  to/TO
  (GPE Biloxi/NNP)
  ,/,
  (GPE Mississippi/NNP)
  ./.)


In [49]:
all_tweet_locations = []

for tweet in tweet_text_list:
    matches = []
    try:
        ne_tree = ne_chunk(nltk.pos_tag(word_tokenize(tweet)))    
        for tag in tree2conlltags(ne_tree):
            if 'GPE' in tag[2]:
                matches.append(tag[0])
        all_tweet_locations.append(matches)
    except:
        all_tweet_locations.append(matches)
        pass
    
        
        
#             print tag
            
#     all_tweet_locations.append(' '.join(matches))
    
    
#     try:
#         if dparser.parse(tweet,fuzzy=False):
#             all_tweet_dates.append(dparser.parse(tweet,fuzzy=False))
#         else:
#             all_tweet_dates.append([])
#     except:
#         all_tweet_dates.append([])
        

In [16]:
# for i,loc in enumerate(all_tweet_locations):
#     if len(loc) > 0:
#         print i, ' ', loc, tweet_text_list[i]

In [11]:
nlp = spacy.load('en')
doc = nlp(u'I am going to protest in Dayton,Ohio')
for ent in doc.ents:
    print(ent.label_, ent.text)


(u'GPE', u'Dayton')
(u'GPE', u'Ohio')


In [75]:
tweet_entities = []

tag_search = ['ORG','LOC','GPE']

for tweet in tweet_text_list:
    matches = []
#     try:
    doc = nlp(tweet.decode('latin-1'))
    for ent in doc.ents:
        matches.append((ent.label_, ent.text))
    tweet_entities.append(matches)
#     except:
#         tweet_entities.append(matches)
#         pass

In [96]:
dates_dont_include = ['year old','old','decades', 'year olds', 'ago', 'ryear', 'calendar', '1 or 2','weeks','year', 'years', 'days', 'old', 'olds', '80s', '90s', '%', '1k', '2k', '3k', '4k', '5k', '6k', '7k', '8k', '9k', '10k', '0k', 'weeks', 'months', 'more', 'anniversary', 'yr', '@', 'past', 'circa', 'last', 'later', 'yesterday', 'yall', 'you', 'up to', 'illegal', '50/50', 'Free', '100 Days', 'many', '#', 'ly', 'Rs', 'up to']
dates_include = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December', '1/', '2/', '3/', '4/', '5/', '6/', '7/', '8/', '9/', '10/', '11/', '12/', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', '2017/', '2018/', '2019/', '01/', '02/', '03/', '04/', '05/', '06/', '07/', '08/', '09/', '10/', '11/', '12/', '7am', '8am', '9am', '10am', '11am', '12am', '1pm', '2pm', '3pm', '4pm', '5pm', '6pm', '7pm', '8pm', '9pm', '10pm', '11pm', '12am', '7:00 AM', '8:00 AM', '9:00 AM', '10:00 AM', '11:00 AM', '12:00 PM', '1:00 PM', '2:00 PM', '3:00 PM', '4:00 PM', '5:00 PM', '6:00 PM', '7:00 PM', '8:00 PM', '9:00 PM', '10:00 PM', '11:00 PM', '12:00 AM', '7:00AM', '8:00AM', '9:00AM', '10:00AM', '11:00AM', '12:00PM', '1:00PM', '2:00PM', '3:00PM', '4:00PM', '5:00PM', '6:00PM', '7:00PM', '8:00PM', '9:00PM', '10:00PM', '11:00PM', '12:00AM', '7 AM', '8 AM', '9 AM', '10 AM', '11 AM', '12 PM', '1 PM', '2 PM', '3 PM', '4 PM', '5 PM', '6 PM', '7 PM', '8 PM', '9 PM', '10 PM', '11 PM', '12 AM', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'Today', 'Tomorrow', 'Day after Tomorrow', 'Day after ', 'Everyday this week', 'Every day this week', 'Everyday this month', 'every day this month', 'next 2', 'next 3', 'next 4', 'next 5', 'next 6', 'next 7', 'next 8', 'next 9', 'next 10', 'this week', 'next week', 'two weeks', 'three weeks', 'four weeks', '2 weeks', '3 weeks', '4 weeks', 'this weekend', 'next Monday', 'next Tuesday', 'next Wednesday', 'next Thursday', 'next Friday', 'next Saturday', 'next Sunday', 'this Mon', 'this Tue', 'this Wed', 'this Thur', 'this Fri', 'This Sat', 'This Sun', 'ThisWeek']

include_regex = re.compile(r'\b(' + '|'.join(dates_include) + r')', re.IGNORECASE)
exclude_regex = re.compile(r'^(' + '|'.join(dates_dont_include) + r').*', re.IGNORECASE)

In [114]:
date_list = []

for i,tweet in enumerate(tweet_entities):
    for tup in tweet:
        if tup[0] == u'DATE':
            if re.findall(exclude_regex,tup[1]):
                pass
            elif re.findall(include_regex,tup[1]) and 'weeks' not in tup[1] and 'months' not in tup[1] and 'old' not in tup[1]:
                try:
                    date_list.append((i,tup[1]))
                except:
                    pass
#             print tup, tweet_text_list[i]
            

In [124]:
dates = []

doc = nlp("ICYMI:  Health care town hall reminder (Tuesday, March 21!) and legislative update.  https://t.co/GLA83W3uyA #mnleg".decode('latin-1'))
for ent in doc.ents:
    matches.append((ent.label_, ent.text))
    if ent.label_ == u'DATE':
        if re.findall(exclude_regex,ent.text):
            pass
        elif re.findall(include_regex,ent.text) and 'weeks' not in ent.text and 'months' not in ent.text and 'old' not in ent.text:
            try:
                dates.append(ent.text)
            except:
                pass
            
print dates
                                    

[u'Tuesday, March 21']


[]

In [19]:
# for date_tup in date_list:
#     print date_tup[1], tweet_text_list[date_tup[0]]

In [48]:
states = ['IA', 'KS', 'UT', 'VA', 'NC', 'NE', 'SD', 'AL', 'ID', 'FM', 'DE', 'AK', 'CT', 'PR', 'NM', 'MS', 'PW', 'CO', 'NJ', 'FL', 'MN', 'VI', 'NV', 'AZ', 'WI', 'ND', 'PA', 'OK', 'KY', 'RI', 'NH', 'MO', 'ME', 'VT', 'GA', 'GU', 'AS', 'NY', 'CA', 'HI', 'IL', 'TN', 'MA', 'OH', 'MD', 'MI', 'WY', 'WA', 'OR', 'MH', 'SC', 'IN', 'LA', 'MP', 'DC', 'MT', 'AR', 'WV', 'TX','Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming','Ia','Ks','Ut','Va','Nc','Ne','Sd','De','Ak','Ct','Nj','Fl','Mn','Vi','Nv','Az','Wi','Pa',
                          'Vt','Ga','Al','Ny','Ca','Il','Tn','Md','Wy','Wa','Mh','Sc']
state_regex = re.compile(r'\b(' + '|'.join(states) + r')\b')

In [20]:
cities = pd.read_csv(os.path.join(os.pardir, 'data/static_data', 'Top5000Population.csv'))
cities['full_city'] = '\"' + cities['city'].str.rstrip() + ', ' + cities['state'] + '\"'
city_list = list(cities['city'].str.rstrip())
city_regex = re.compile(r'\b(' + '|'.join(city_list) + r')\b', re.IGNORECASE)

In [49]:
all_tweet_cities = []

for tweet in tweet_text_list:
    matches = []
    try:
        if city_regex.match(tweet):
            for match in re.findall(city_regex, tweet):
                if match != 'White House' and match != 'Liberal' and match != 'Perry' and match != 'Price':
                    matches.append(match)
            all_tweet_cities.append(matches)
        else:
            all_tweet_cities.append([])
    except:
        all_tweet_cities.append([])

In [18]:
# for i, rec in enumerate(all_tweet_cities):
#     if len(rec) > 0:
#         print i,  rec, tweet_text_list[i]

In [50]:
all_tweet_states = []

for tweet in tweet_text_list:
    matches = []
    try:
        if state_regex.match(tweet):
            for match in re.findall(state_regex, tweet):
                if match != 'MT':
                    matches.append(match)
            all_tweet_states.append(matches)
        else:
            all_tweet_states.append([])
    except:
        all_tweet_states.append([])

In [3]:
# for i, rec in enumerate(all_tweet_states):
#     if len(rec) > 0:
#         print i,  rec, tweet_text_list[i]
        

In [4]:
sunlight.config.API_KEY = 'thisisakey'
legislators = congress.all_legislators_in_office()

In [8]:
print legislators[0]

{u'last_name': u'Strange', u'state_name': u'Alabama', u'office': u'G-12 Dirksen Senate Office Building', u'fax': None, u'thomas_id': u'', u'first_name': u'Luther', u'middle_name': None, u'district': None, u'senate_class': 2, u'in_office': True, u'state': u'AL', u'term_end': u'2021-01-03', u'crp_id': u'', u'oc_email': None, u'party': u'R', u'fec_ids': [u'S8AL00308'], u'website': None, u'lis_id': u'S392', u'leadership_role': None, u'govtrack_id': u'412734', u'bioguide_id': u'S001202', u'birthday': u'1953-03-01', u'term_start': u'2017-02-09', u'nickname': None, u'contact_form': None, u'ocd_id': u'ocd-division/country:us/state:al', u'phone': u'202-224-4124', u'gender': u'M', u'title': u'Sen', u'name_suffix': None, u'twitter_id': u'SenatorStrange', u'chamber': u'senate', u'state_rank': u'junior'}


In [7]:
set().union(*(d.keys() for d in legislators))

{u'bioguide_id',
 u'birthday',
 u'chamber',
 u'contact_form',
 u'crp_id',
 u'district',
 u'facebook_id',
 u'fax',
 u'fec_ids',
 u'first_name',
 u'gender',
 u'govtrack_id',
 u'icpsr_id',
 u'in_office',
 u'last_name',
 u'leadership_role',
 u'lis_id',
 u'middle_name',
 u'name_suffix',
 u'nickname',
 u'oc_email',
 u'ocd_id',
 u'office',
 u'other_names',
 u'party',
 u'phone',
 u'senate_class',
 u'state',
 u'state_name',
 u'state_rank',
 u'term_end',
 u'term_start',
 u'thomas_id',
 u'title',
 u'twitter_id',
 u'votesmart_id',
 u'website',
 u'youtube_id'}

In [41]:
twitter_legs = []
leg_names = []

for leg in legislators:
    if leg.get('twitter_id',None):
        twitter_legs.append('@'+leg.get('twitter_id',None))
    
    name1 = leg.get('first_name',None) + ' ' + leg.get('last_name',None)
    name2 = leg.get('title',None) + ' ' + leg.get('last_name',None)
    name3 = leg.get('title',None) + ' ' + leg.get('first_name',None) + ' ' + leg.get('last_name',None)
    if leg.get('title',None) == 'Sen':
        name4 = 'Senator ' + leg.get('last_name',None)
        name5 = 'Senator ' + leg.get('first_name',None) + ' ' + leg.get('last_name',None)
        name6 = ''
        name7 = ''
    elif leg.get('title',None) == 'Rep': 
        name4 = 'Representative ' + leg.get('last_name',None)
        name5 = 'Representative ' + leg.get('first_name',None) + ' ' + leg.get('last_name',None)
        if leg.get('gender',None) == 'M':
            name6 = 'Congressman ' + leg.get('last_name',None)
            name7 = 'Congressman ' + leg.get('first_name',None) + ' ' + leg.get('last_name',None)
        elif leg.get('gender',None) == 'F':
            name6 = 'Congresswoman ' + leg.get('last_name',None)
            name7 = 'Congresswoman ' + leg.get('first_name',None) + ' ' + leg.get('last_name',None)
        else:
            name6 = ''
            name7 = ''
    else:
        name3 = ''
        name4 = ''
        name5 = ''
        name6 = ''
        name7 = ''    
        
    leg_names.append(name1)
    leg_names.append(name2)
    leg_names.append(name3)
    leg_names.append(name4)
    leg_names.append(name5)
    leg_names.append(name6)
    leg_names.append(name7)

leg_names = list(set(leg_names))

In [42]:
leg_name_regex = re.compile(r'\b(' + '|'.join(leg_names[1:]) + r')\b', re.IGNORECASE)
leg_twitter_regex = re.compile(r'(' + '|'.join(twitter_legs) + r')\b', re.IGNORECASE)


In [31]:
all_tweet_legs = []

for tweet in tweet_text_list:
    matches = []
    try:
        if leg_name_regex.match(tweet):
            for match in re.findall(leg_name_regex, tweet):
                matches.append(match)
            all_tweet_legs.append(matches)
        else:
            all_tweet_legs.append([])
    except:
        all_tweet_legs.append([])

In [32]:
for i, rec in enumerate(all_tweet_legs):
    if len(rec) > 0:
        print i,  rec, tweet_text_list[i]
        

sys... https://t.co/twg1SNLbbe Slip of the tongue, but revealing anyway
sys... https://t.co/MVWnK1wank
sys... https://t.co/fBlF9mOTPs Yes, he really said this.
195 ['Paul Ryan'] Paul Ryan: 'We're not going to give up on destroying the health care system' https://t.co/WmSv5aU6Ev
249 ['Paul Ryan'] Paul Ryan: Been dreaming of screwing old folks since my kegger days - so now we can give tax breaks to super Rich https://t.co/KKjJivdCKA
334 ['Paul Ryan'] Paul Ryan:"Healthcare as a right is 2 give government 2 much power over our lives" U're right Paul,give it power 2 pay the bill w/ our taxes
sys... https://t.co/rCsgX0L9G8 via @YouTube
523 ['Paul Ryan'] Paul Ryan wants to kill Medicare and give our money to Wall Street. My plan calls for saving Medicare. Hands off!��_ https://t.co/3ePTGegk4I
562 ['Paul Ryan'] Paul Ryan: 'We're not going to give up on destroying the health care system' https://t.co/dFN78YKrpm
644 ['Nancy Pelosi'] Nancy Pelosi Gets Roasted on Twitter After Asking People to Giv

In [43]:
all_tweet_legs_twitter = []

for tweet in tweet_text_list:
    matches = []
    try:
        if leg_twitter_regex.match(tweet):
            for match in re.findall(leg_twitter_regex, tweet):
                matches.append(match)
            all_tweet_legs_twitter.append(matches)
        else:
            all_tweet_legs_twitter.append([])
    except:
        all_tweet_legs_twitter.append([])

In [44]:
for i, rec in enumerate(all_tweet_legs_twitter):
    if len(rec) > 0:
        print i,  rec, tweet_text_list[i]

14 ['@SenSanders'] @SenSanders Medicare is not free and neither is Medicaid. "Give me ------- or give me -----! When I worked in my first 11 years I had none!
26 ['@SpeakerRyan'] @SpeakerRyan $1200/mo for health insurance for a 64 year old is what Obamacare is charging. Give up the govt employees plans to choose from
36 ['@SpeakerRyan'] @SpeakerRyan  You are taking from the poor to give more to the rich. You do not care because the American people pay your health insurance.
38 ['@SpeakerRyan', '@RepKevinBrady'] @SpeakerRyan @RepKevinBrady stop wasting money on useless programs and give us affordable great healthcare.
92 ['@SpeakerRyan', '@RepKevinBrady'] @SpeakerRyan @RepKevinBrady shouldnt you be focused on the healthcare bill or did you give up
93 ['@SpeakerRyan'] @SpeakerRyan We've already paid 4 our healthcare many times over.  All the money  you'd rather give 2 corporations, the wall, the military.
97 ['@keithellison'] @keithellison please, for the love of all that is holy, put fo

In [13]:

# state_list = list(cities['state'].str.rstrip())
                 
# city_state_tuples = zip(city_list, state_list)
# print city_list[:10]
# print city_state_tuples[:10]
# # print city_list.index('Jersey City, NJ')